# Dynamic Breakout II Strategy

Decide look-back period based on rate of change of volatility
Make decisions based on highest high and lowest low from look back period (Bollinger bands as indictor)

Original dynamic breakout, volatility component changes lookback period
New original dynamic breakout includes Bollinger Bands and adjusts number of look back using market volatility

Start, set lookback to 20, change in proportion to changes in volatilty (30 day std closing price) (restrict to range of 20 to 60)
$$deltavol = \frac{todayvol-yesterdayvol}{todayvol}$$
$$Lookback = round(numdays*(1+deltavol))$$

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_colwidth', None)
from decimal import ROUND_HALF_UP, Decimal
from pandas import api
from statsmodels.api import OLS
import random
import statsmodels.api as sm
from ta.volatility import BollingerBands
from tqdm.auto import tqdm
tqdm.pandas()


In [2]:
train_stock_prices = pd.read_csv('ds/train_files/stock_prices.csv')
supplemental_stock_prices = pd.read_csv('ds/supplemental_files/stock_prices.csv')
data_stock_prices = pd.concat([train_stock_prices,supplemental_stock_prices],ignore_index=True)

In [3]:
def calc_adjusted_close(df):
    df = df.sort_values("Date",ascending=False)
    df.loc[:,"cummulative_adjustment_factor"] = df["AdjustmentFactor"].cumprod()
    df.loc[:,"adjusted_close"] = (df["cummulative_adjustment_factor"]*df["Close"]).map(lambda x: float(Decimal(str(x)).quantize(Decimal("0.1"),rounding=ROUND_HALF_UP)))
    df.loc[:,"adjusted_open"] = (df["cummulative_adjustment_factor"]*df["Open"]).map(lambda x: float(Decimal(str(x)).quantize(Decimal("0.1"),rounding=ROUND_HALF_UP)))
    df.loc[:,"adjusted_high"] = (df["cummulative_adjustment_factor"]*df["High"]).map(lambda x: float(Decimal(str(x)).quantize(Decimal("0.1"),rounding=ROUND_HALF_UP)))
    df.loc[:,"adjusted_low"] = (df["cummulative_adjustment_factor"]*df["Low"]).map(lambda x: float(Decimal(str(x)).quantize(Decimal("0.1"),rounding=ROUND_HALF_UP)))
    df = df.sort_values("Date")
    df.loc[df["adjusted_close"]==0,"adjusted_close"] = np.nan
    df.loc[:,"adjusted_close"] = df.loc[:,"adjusted_close"].ffill()
    return df

In [4]:
def calc_adjusted_volume(df):
    df.loc[:,"adjusted_volume"] = (df["Volume"]/df["cummulative_adjustment_factor"]).map(lambda x: float(Decimal(str(x)).quantize(Decimal("0.1"),rounding=ROUND_HALF_UP)))
    df = df.sort_values("Date")
    df.loc[df["adjusted_volume"]==0,"adjusted_volume"] = np.nan
    df.loc[:,"adjusted_volume"] = df.loc[:,"adjusted_volume"].ffill()
    return df

In [5]:

def calc_spread_return_sharpe(df: pd.DataFrame, portfolio_size: int = 200, toprank_weight_ratio: float = 2) -> float:
    """
    Args:
        df (pd.DataFrame): predicted results
        portfolio_size (int): # of equities to buy/sell
        toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
    Returns:
        (float): sharpe ratio
    """
    def _calc_spread_return_per_day(df, portfolio_size, toprank_weight_ratio):
        """
        Args:
            df (pd.DataFrame): predicted results
            portfolio_size (int): # of equities to buy/sell
            toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
        Returns:
            (float): spread return
        """
        assert df['Rank'].min() == 0
        assert df['Rank'].max() == len(df['Rank']) - 1
        weights = np.linspace(start=toprank_weight_ratio, stop=1, num=portfolio_size)
        #Target is the rate of change 
        purchase = (df.sort_values(by='Rank')['Target'][:portfolio_size] * weights).sum() / weights.mean()
        short = (df.sort_values(by='Rank', ascending=False)['Target'][:portfolio_size] * weights).sum() / weights.mean()
        return purchase - short

    buf = df.groupby('Date').apply(_calc_spread_return_per_day, portfolio_size, toprank_weight_ratio)
    sharpe_ratio = buf.mean() / buf.std()
    print(buf.mean())
    print(buf.std())
    buf.plot()
    return sharpe_ratio

In [6]:
def create_bands(df):
    df["upper_band"] = df["moving_average"] + (2*df["moving_std"])
    df["lower_band"] = df["moving_average"] - (2*df["moving_std"])
    df["difference_upper"] = df['adjusted_close']-df["upper_band"]
    df["difference_lower"] = df['lower_band'] - df["adjusted_close"]
    return df

In [7]:
def create_deciles(df):
    df = df.sort_values("difference_upper",ascending=False)
    to_buy_index = df.index[:200]
    df = df.sort_values("difference_lower",ascending=False)
    to_short_index = df.index[:200]
    df.loc[to_buy_index,"Quarter"] = 0
    df.loc[to_buy_index,"Abs"] = df.loc[to_buy_index,"difference_upper"]
    df.loc[to_short_index,"Quarter"] = 2
    df.loc[to_short_index,"Abs"] = df.loc[to_short_index,"adjusted_close"]-df.loc[to_short_index,"lower_band"]
    df["Quarter"]=df["Quarter"].fillna(1)
    df = df.sort_values(["Quarter","Abs"],ascending=[True,False])
    return df

In [8]:
def calc_rolling(df,t):
    for y in df["num_days"].unique():
        y=y.astype('int')
        if(y>0):
            if(t=='mean'):
                df[y] = df["adjusted_close"].rolling(y).mean()
            else:
                df[y] = df["adjusted_close"].rolling(y).std()
    return df

In [9]:
def create_features(df):
    df = df.copy()
    df["Date"] = pd.to_datetime(df["Date"])
    df = df.drop(["RowId"],axis=1)
    df = df[df["Date"]!="2020-10-01"]
    df = df.groupby("SecuritiesCode").apply(calc_adjusted_close).reset_index(drop=True).sort_values(["Date","SecuritiesCode"]).reset_index(drop=True)
    df = df.groupby("SecuritiesCode").apply(calc_adjusted_volume).reset_index(drop=True).sort_values(["Date","SecuritiesCode"]).reset_index(drop=True)
    df["today_vol"] = df.groupby("SecuritiesCode")["adjusted_close"].rolling(30).std().reset_index(0,drop=True)
    df["yesterday_vol"] = df.groupby("SecuritiesCode")["adjusted_close"].apply(lambda x: x.shift(1).rolling(30).std())
    df["delta_vol"] = df.groupby("SecuritiesCode").apply(lambda x: (x["today_vol"]-x["yesterday_vol"])/x["today_vol"]).reset_index(0,drop=True)
    df["num_days"] = df.groupby("SecuritiesCode").apply(lambda x: round(20*(1 + x["delta_vol"]))).reset_index(0,drop=True)
    df.loc[df["num_days"]>60,"num_days"] = 60
    df.loc[df["num_days"]<20,"num_days"] = 20
    df=df.groupby("SecuritiesCode").apply(lambda x:calc_rolling(x,'mean'))
    df[np.nan]=np.nan
    df['moving_average']=df.apply(lambda x: x[x['num_days']],axis=1)
    df =df.groupby("SecuritiesCode").apply(lambda x: calc_rolling(x,'std'))
    df['moving_std']=df.apply(lambda x: x[x['num_days']],axis=1)
    df = df.drop(labels=df['num_days'].unique(),axis=1)
    df = df.groupby("SecuritiesCode").apply(create_bands)
    df = df.groupby("Date").apply(create_deciles).reset_index(0,drop=True)
    df=df.dropna(subset=['moving_average'])
    df["Rank"] = df.groupby(["Date"])["SecuritiesCode"].transform(lambda x: np.linspace(0,x.count()-1,x.count()))
    return df

In [11]:
df = create_features(train_stock_prices)

KeyboardInterrupt: 

In [ ]:
calc_spread_return_sharpe(df)

In [ ]:
tmp["adjusted_close"].rolling(28).std()

In [ ]:
test

In [ ]:
test = create_features(train_stock_prices)
calc_spread_return_sharpe(test)

In [ ]:
#This is factor 2
test = create_features(train_stock_prices)
calc_spread_return_sharpe(test)

In [ ]:
#41
test = create_features(train_stock_prices)
calc_spread_return_sharpe(test)